# Uczenie maszynowe: Lab3

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report, accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.exceptions import ConvergenceWarning
from sklearn.ensemble import GradientBoostingClassifier
import warnings
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectFromModel, SelectKBest, f_classif
from sklearn.ensemble import BaggingClassifier
from sklearn.datasets import load_digits

### Analiza modelu klasyfikacyjnego
* wziąć konkretne dane, podzielić na część treningową i testową lub uczyć wg. schematu walidacji
krzyżowej
* sprawdzić jakie są hiperparametry danego modelu i znaleźć optymalne
* uczyć parametry modelu z danych, po skończonej nauce je wypisać

In [ ]:
warnings.filterwarnings('ignore', category=ConvergenceWarning)

data = load_digits()

X = data.data 
y = data.target 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) # test_size -> 80% training and 20% test
# random_state = 42 (służy jako ziarne losowości, aby wyniki były powtarzalne), natomiast jak wartość jest ta sama za każdym razem
# to mamy ten sam podział

print ("\nX_test = ", X_test.shape)
print ("Y_test = ", y_test.shape,)
print ("X_train = ", X_train.shape)
print ("Y_train = ", y_train.shape)

# Walidacja krzyżowa k - fold
# Dzielimy na k równych podzbiorów (w których każdy z podzbiorów raz występuje jako zbiór uczący, a pozostała, połączona 
# część zbioru jest wykorzystywana jako zbiór testowy)

k = 5
cv = StratifiedKFold(n_splits=k) # -> schemat walidacji krzyżowej

In [ ]:
def evaluate_model(X_train, X_test, Y_train, Y_test, model, param_grid, cv):
    cross_val_scores = cross_val_score(model, X_train, Y_train, cv=cv, scoring='accuracy') # metryka oceny -> dokładność

    # Walidacja krzyżowa - technika trenowania i testowania modelu na różnych podzbiorach danych
    # Dostajemy procentowo jak dobrze model działa tzn jaka jest dokładność
    # na danym podzbiorze danych
    print("\nWyniki walidacji krzyżowej: \n", cross_val_scores)
    print("Średni wynik walidacji krzyżowej: ", cross_val_scores.mean()) 

    # Grid Search -> wyszukanie najlepszych hiperparametrów
    # param grid -> określamy jakie hiperparametry chcemy przetestować
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=cv, scoring='accuracy')
    grid_search.fit(X_train, Y_train)

    # Najlepsze hiperparametry
    best_params = grid_search.best_params_
    print(f"Najlepsze hiperparametry: {best_params}")

    # Trenowanie modelu z najlepszymi hiperparametrami
    best_model = grid_search.best_estimator_
    best_model.fit(X_train, Y_train)

    # Predykcje i ocena na zbiorze testowym
    y_pred = best_model.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    print(f"Dokładność na zbiorze testowym: {accuracy:.4f}")
    print("Raport klasyfikacji:\n", classification_report(y_test, y_pred, zero_division=0))

    # Wypisanie końcowych parametrów modelu
    print("Końcowe parametry modelu:")
    print(best_model.get_params())

### 1. Naiwny klasyfikator Bayesa
Prosty, ale potężny algorytm uczenia maszynowego stosowany głównie do problemów klasyfikacji. Nazwa "naiwny" pochodzi od założenia, że wszystkie cechy (atrybuty) w zbiorze danych są niezależne od siebie, co rzadko jest prawdą w rzeczywistych zastosowaniach.

In [ ]:
model = GaussianNB()
param_grid = {
    'var_smoothing': [1e-9, 1e-8, 1e-7, 1e-6, 1e-5]  # Dodaje niewielką wartość do wariancji, aby zapobiec dzieleniu przez zero
}

evaluate_model(X_train, X_test, y_train, y_test, model, param_grid, cv)

### 2. Model kNN (model k najbliższych sąsiadów)
Ma za zadnie znaleźć k sąsiadów, do których klasyfikowane obiekty są najbliższe dla wybranej metryki (np. Euklidesa), a następnie określa
wynik klasyfikacji na podstawie większości głosów tych najbliższych k sąsiadów, biorąc pod uwagę, która klasa jest reprezentowana największą liczbę razy w grupie k najbliższych sąsiadów

In [ ]:
model = KNeighborsClassifier()
param_grid = {
    'n_neighbors': [3, 5, 7, 9], # Liczba najbliższych sąsiadów
    'weights': ['uniform', 'distance'], # Waga dla sąsiadów, uniform - wszyscy sąsiedzi mają taką samą wagę, distance - im bliżej tym większa waga
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'], # Algorytm używany do wyszukiwania najbliższych sąsiadów.
    'metric': ['euclidean', 'manhattan', 'minkowski'] # Metryka używana do obliczania odległości
}

# Poprawia wydajność modeli wrażliwych na skalowanie danych
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

evaluate_model(X_train, X_test, y_train, y_test, model, param_grid, cv)

### 3. Drzewo decyzyjne
Model drzewa decyzyjnego jest strukturą drzewa, gdzie węzły reprezentują decyzje na podstawie wartości cech, a gałęzie przedstawiają możliwe wyniki tych decyzji. Liście drzewa reprezentują klasy (w przypadku klasyfikacji) lub wartości przewidywane (w przypadku regresji).

#### Jak działa drzewo decyzyjne?
##### Budowanie drzewa:

- Algorytm zaczyna od całego zbioru danych jako korzenia drzewa.
- W każdym węźle wybiera najlepszą cechę i wartość progu do podziału danych, maksymalizując czystość podzbiorów po podziale.
- Proces ten jest rekurencyjnie powtarzany dla każdego podzbioru, tworząc gałęzie drzewa, aż do spełnienia kryterium zatrzymania (np. maksymalna głębokość drzewa, minimalna liczba próbek w liściu).
Czystość podzbiorów:

#### Czystość podzbiorów 
- Po podziale jest oceniana za pomocą miar takich jak indeks Giniego czy entropia. Celem jest maksymalizacja czystości podzbiorów, tj. dążenie do sytuacji, gdzie podzbiory zawierają próbki głównie jednej klasy.

In [ ]:
model = DecisionTreeClassifier()
param_grid = {
    'criterion': ['gini', 'entropy'], # Kryterium oceny jakości podziału. indeks Giniego/Entropia (używa entropii do oceny czystości podziału)
    'max_depth': [None, 10, 20, 30], # Maksymalna głębokość drzewa, None - brak ograniczenia
    'min_samples_split': [2, 5, 10], # Minimalna liczba próbek wymagana do podziału węzła
    'min_samples_leaf': [1, 2, 4], # Minimalna liczba próbek wymagana w liściu węzła
    'splitter': ['best', 'random'], # Strategia podziału węzła 'best' - najlepszy podział, 'random' - losowy podział
    'max_features': ['sqrt', 'log2'], # Maksymalna liczba cech branych pod uwagę przy poszukiwaniu najlepszego podziału
}

evaluate_model(X_train, X_test, y_train, y_test, model, param_grid, cv)

### 4. Regresja logistyczna
Algorytmem klasyfikacyjnym, który jest używany do przewidywania binarnych wyników (0 lub 1). Model ten oblicza prawdopodobieństwo przynależności próbki do określonej klasy, stosując funkcję logistyczną (sigmoidę).

In [ ]:
model = LogisticRegression(max_iter=1000) # max_iter - maksymalna liczba iteracji
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100], # Odwrotność siły regularyzacji, mniejsza wartość oznacza silniejszą regularyzację
    'solver': ['liblinear', 'saga'], # Algorytm używany do optymalizacji
    # 'penalty': ['l1', 'l2', 'elasticnet'], # Rodzaj regularyzacji (None - break regularyzacji)
}

# Poprawia wydajność modeli wrażliwych na skalowanie danych
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

evaluate_model(X_train, X_test, y_train, y_test, model, param_grid, cv)

### 5. Model LDA (Linear Discriminant Analysis)
Jest to technika stosowana w statystyce, rozpoznawaniu wzorców i uczeniu maszynowym do znajdowania liniowej kombinacji cech, która najlepiej oddziela dwie lub więcej klas.

In [ ]:
model = LinearDiscriminantAnalysis()
param_grid = [
    {'solver': ['svd']},
    {'solver': ['lsqr', 'eigen'], 'shrinkage': ['auto', 0.1, 0.5, 1.0]}
]

evaluate_model(X_train, X_test, y_train, y_test, model, param_grid, cv)

### 6. Model MLP (Prosta sieć neuronowa)
Jest rodzajem sztucznej sieci neuronowej stosowanej w uczeniu nadzorowanym do rozwiązywania problemów klasyfikacyjnych. Jest to pełnoprawny klasyfikator, który uczy się na podstawie danych wejściowych i dostarcza predykcje klas.

In [ ]:
model = MLPClassifier(max_iter=100)
param_grid = {
    'hidden_layer_sizes': [(50,), (100,), (50, 50)], # Wiele warstw ukrytych
    'activation': ['tanh', 'relu'], # Funkcja aktywacji dla warstw ukrytych
    'solver': ['sgd', 'adam'], # Optymalizator
    'alpha': [0.0001, 0.1], # Parametr regularyzacji (L2)
    'learning_rate': ['constant', 'adaptive'], # Plan uczenia
}

# Poprawia wydajność modeli wrażliwych na skalowanie danych
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

evaluate_model(X_train, X_test, y_train, y_test, model, param_grid, cv)

### 7. Model SVM (Support Vector Machine)
Algorytm uczenia maszynowego używany głównie do zadań klasyfikacyjnych, ale może być również stosowany do regresji (Support Vector Regression). SVM znajduje hiperpłaszczyznę, która najlepiej rozdziela dane w przestrzeni cech.

In [ ]:
model = SVC()
param_grid = {
    'C': [0.1, 1, 10, 100], # Parametr regularyzacji. Kontroluje kompromis między maksymalizacją marginesu a minimalizacją błędu klasyfikacji.
    'gamma': [1, 0.1, 0.01, 0.001], # Parametr funkcji jądrowej RBF (Radial Basis Function). 
    # Wyższa wartość oznacza większy wpływ próbek treningowych.
    'kernel': ['linear', 'rbf'] # Funkcja jądrowa
}

# Poprawia wydajność modeli wrażliwych na skalowanie danych
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

evaluate_model(X_train, X_test, y_train, y_test, model, param_grid, cv)

### 8. Bagging
Metoda zespołowa (ensemble), która poprawia stabilność i dokładność algorytmów uczenia maszynowego poprzez zmniejszenie wariancji. Metoda polega na trenowaniu wielu modeli bazowych na różnych losowo wybranych podzbiorach danych treningowych, a następnie łączeniu ich predykcji.

In [ ]:
base_model = DecisionTreeClassifier()
model = BaggingClassifier(estimator=base_model, n_estimators=100, random_state=42) # defaultowo estimator to DecisionTreeClassifier
param_grid = {
    'n_estimators': [50, 100, 200], # Liczba modeli bazowych (drzew decyzyjnych) w ensemble.
    'max_features': [0.5, 0.7, 1.0], # Maksymalna liczba cech branych pod uwagę przy poszukiwaniu najlepszego podziału
    'max_samples': [0.5, 1.0], # Liczba próbek używanych do trenowania każdego modelu bazowego
    'bootstrap': [True, False], # Czy próbki są losowane z powtórzeniami
    'bootstrap_features': [True, False], # Czy cechy są losowane z powtórzeniami
}
evaluate_model(X_train, X_test, y_train, y_test, model, param_grid, cv)

### 9. Boosting
Technika zespołowa, która tworzy silny model predykcyjny poprzez łączenie słabych modeli bazowych, zwykle drzew decyzyjnych. Metoda ta działa iteracyjnie, poprawiając błędy poprzednich modeli.

In [ ]:
model = GradientBoostingClassifier()
param_grid = {
    'n_estimators': [50, 100, 200], # Liczba drzew decyzyjnych
    'learning_rate': [0.01, 0.1, 1, 10], # Współczynnik uczenia
    'max_depth': [3, 5, 7], # Maksymalna głębokość drzewa
}
evaluate_model(X_train, X_test, y_train, y_test, model, param_grid, cv)

### 10. Lasy losowe
Metoda zespołowa (ensemble) polegająca na budowie wielu drzew decyzyjnych podczas treningu i wyjściu klasy będącej trybem klas (dla klasyfikacji) lub średnią predykcji (dla regresji) poszczególnych drzew. Random forest łączy prostotę drzew decyzyjnych z możliwościami agregacji w celu uzyskania lepszej wydajności i stabilności predykcji.

In [ ]:
model = RandomForestClassifier(random_state=42)
param_grid = {
    'n_estimators': [50, 100, 200],  # Liczba drzew w lesie
    'max_features': ['sqrt', 'log2'],  # Liczba cech do rozważenia przy podziale
    'max_depth': [None, 10, 20, 30],  # Maksymalna głębokość drzew
    'min_samples_split': [2, 5, 10],  # Minimalna liczba próbek potrzebnych do podziału węzła
    'min_samples_leaf': [1, 2, 4],  # Minimalna liczba próbek w liściu
    'bootstrap': [True, False]  # Czy losować próbki z powtórzeniami
}
evaluate_model(X_train, X_test, y_train, y_test, model, param_grid, cv)

### Uczenie nienadzorowane

### Inżynieria cech

Selekcja cech (różne algorytmy).

In [ ]:
# Dane
data = load_digits()

X = data.data 
y = data.target 

# Trenowanie modelu
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standaryzacja danych
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Tworzenie modelu Random Forest
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

# Trening modelu bez selekcji cech
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Dokładność na zbiorze testowym (bez selekcji): {accuracy:.4f}")
print("Raport klasyfikacji (bez selekcji):\n", classification_report(y_test, y_pred))

# Selekcja cech na podstawie znaczenia cech
selector = SelectFromModel(rf, prefit=True)
X_train_rf = selector.transform(X_train)
X_test_rf = selector.transform(X_test)

# Trening modelu z selekcją cech (znaczenie cech)
model.fit(X_train_rf, y_train)
y_pred_rf = model.predict(X_test_rf)
accuracy_rf = accuracy_score(y_test, y_pred_rf)
print(f"Dokładność na zbiorze testowym (znaczenie cech): {accuracy_rf:.4f}")
print("Raport klasyfikacji (znaczenie cech):\n", classification_report(y_test, y_pred_rf))

# Wybieranie najlepszych cech za pomocą testu ANOVA
selector = SelectKBest(score_func=f_classif, k=2)
X_train_best = selector.fit_transform(X_train, y_train)
X_test_best = selector.transform(X_test)

# Trenowanie modelu po selekcji cech
model.fit(X_train_best, y_train)
y_pred_best = model.predict(X_test_best)
accuracy_best = accuracy_score(y_test, y_pred_best)
print(f"Dokładność na zbiorze testowym (Univariate Selection): {accuracy_best:.4f}")
print("Raport klasyfikacji (Univariate Selection):\n", classification_report(y_test, y_pred_best))

Redukcja wymiarowości: Metoda PCA

In [ ]:
# Wczytywanie danych
data = load_digits()

X = data.data 
y = data.target 

# Podział danych na zbiór treningowy i testowy
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standaryzacja danych -> każda zmienna ma średnią 0 i wariancję 1 aby zmienne występujące w zbiorze danych były tej samej skali
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Trenowanie modelu bez redukcji wymiarowości
model = RandomForestClassifier(n_estimators=100, random_state=42)

model.fit(X_train, y_train)
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Dokładność na zbiorze testowym (bez redukcji): {accuracy:.4f}")
print("Raport klasyfikacji (bez redukcji):\n", classification_report(y_test, y_pred))

# Redukcja wymiarowości za pomocą PCA
pca = PCA(n_components=2)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

model.fit(X_train_pca, y_train)
y_pred_pca = model.predict(X_test_pca)
accuracy_pca = accuracy_score(y_test, y_pred_pca)
print(f"\nDokładność na zbiorze testowym (PCA): {accuracy_pca:.4f}")
print("Raport klasyfikacji (PCA):\n", classification_report(y_test, y_pred_pca))